In [2]:

#<<<<<<<<<<<<<<<<<<<<#          Simulacion RS        #>>>>>>>>>>>>>>>>>>>># V 1.0


import random, numpy as np

TamañoDeLaMuestra = 500 # Cantidad de sujetos que quiero en la muestra.
QVecesSimulacion = 10 # Cantidad de veces que quiero que se corra la simulacion.
ParticipanObligados = 10 # Estos van a ser los primeros sujetos, que opinan si o si.





In [9]:

#<<<<<<<<<<<<<<<<<<<<#          Simulacion RS        #>>>>>>>>>>>>>>>>>>>># V 1.0


import random, numpy as np

print ('Ingrese el tamaño de la muestra')
TamañoDeLaMuestra = int(input ()) # Cantidad de sujetos que quiero en la muestra.

print ('Ingrese la cantidad de veces que quiere que corra la simulacion')
QVecesSimulacion = int(input ()) # Cantidad de veces que quiero que se corra la simulacion.

print ('Ingrese la cantidad de participantes que al principio opinan si o si')
ParticipanObligados = int(input ()) # Estos van a ser los primeros sujetos, que opinan si o si.


Ingrese el tamaño de la muestra


 20


Ingrese la cantidad de veces que quiere que corra la simulacion


 30


Ingrese la cantidad de participantes que al principio opinan si o si


 30


In [7]:

# Defino a las personas, cada una tiene una opinion inicial, una eleccion (respecto a como se ajusta a la opinion del otro) y una opinion luego de estar expuesta a otras.
# Hay cosas que no van porque lo traje desde la otra simulacion de practica, luego se acomoda.

class Person (object):
    def __init__ (self, opinion):
        self.Oi = opinion
        self.IdeasVistas = [] # Me va a servir para almacenar las ideas a las que haya estado expuesto el sujeto
    
    def __str__ (self):
        try:
            devolver = 'despues defino que quiero que me devuelva'
            return devolver
        except:
            devolver = 'despues defino que quiero que me devuelva'
            return devolver
    
    def set_Oi (self, opinion):
        self.Oi = opinion
    
    def append_IdeasVistas (self, IdeaVista):
        self.IdeasVistas.append (IdeaVista)
            
    def get_Oi (self):
        return self.Oi
    
    def get_IdeasVistas (self):
        return self.IdeasVistas
    
    def clean_IdeasVistas (self):
        self.IdeasVistas = []


In [4]:

# Creo la muestra
Muestra_Completa = {}
for i in range (TamañoDeLaMuestra):
    Muestra_Completa [i] = Person (np.random.uniform (-1,1))


In [4]:

# Corro la simulacion
for i in range (QVecesSimulacion): #Cantidad de veces que quiero que se repita la simulacion
    Primeros10 = 0
    Muestra_Remanente = Muestra_Completa
    Muestra_Participa = {}
    
    while len (Muestra_Remanente) > 0:
        Q_Participan = random.randint(0,len(Muestra_Remanente))
        Participan_Ahora = random.sample (list(Muestra_Remanente), k = Q_Participan)
        for i in Participan_Ahora:
            Muestra_Participa [i] = Muestra_Remanente [i]
            Muestra_Remanente.pop (i)
            if Primeros10 < ParticipanObligados:
                Muestra_Participa [i] #hacer que escriba algo
                Primeros10 +=1
            #else:
                # definir la probabilidad que el resto escriba algo
